# Simple Fully Connected Neural Netowork
DNN 부터가 실질적인 신경망의 형태를 갖춘 신경망에 대한 설명이라고 볼 수 있다. DNN 은 통상적으로 Hidden Layer 가 2 Depth 이상인 인공 신경망을 말하며, 이번 예제에서는 MNIST (손글씨) 데이터를 이러한 DNN 을 사용하여 Classification 하는 예제이다. 

<H3>(1) 데이터 준비</H3>   
Learning Rate (Weight 값을 Update 할때 적용되는 상수), Epoch (Train 반복 횟수) , Batch Size (훈련 한번에 사용하는 데이터 량), Display Step (로그를 얼마 간격으로 출력할지) 를 지정하고, Hidden Layer 1,2 의 Feature 의 수를 정의하고 Data 를 Feed 할 PlaceHolder 를 정의한다.   
여기서 기존의 단순 Linear, Logistc 등의 모델과 다른 점은 Hidden Layer 인데 아래에서 조금 더 설명하도록 한다. 

In [1]:
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import tensorflow as tf

#tensorboard log directoy
logs_path = '/tmp/tensorflow_logs/chap4-fcnn'

# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


<H3>(2) Graph 정의</H3>   
DNN 이기 때문에 기존의 예제보다는 Train 해야할 파라메터들이 좀 많다. Layer 별 Weight 값과 Bias 값인데, Hidden Layer-1, Hidden Layer-2, Out Layer 3개에 대해서 앞뒤 Layer 의 사이즈에 맞춰서 Matrix 사이즈를 정하면 된다.   
그 다음으로 각각의 Layer 구성인데 Neural Network 에서는 각각의 Node 를 Perceptron 이라고 명명하며 ActivationFunc(wx + b) 의 형태를 가지고 있다. 여기서는 Relu 라는 Activiation Function 을 사용하는데 Relu는 앞에서 사용하던 Logistic 과는 다르게 0 ~ 무한대에 수렴하도록 하는 특성을 가지고 있다. 최근에는 대부분의 DNN 에서 Relu 를 많이 사용하는데 그 이유는 Depth 가 깊어 질수록 발생하는 Vanishing Problem 때문이다.   
Relu 는 0~무한대를 지원하기 때문에 신호가 뒤로 갈수로 사라지는 Vanishing Problem 을 예방 할 수 있다.  
아래의 예에서 보면 Hidden Layer 는 2 Level 로 구성되어 있으며, 아웃풋은 10개 (0~9까지 숫자)로 구성되어 있다.   
인풋 - 히든 - 아웃풋을 정의해 보면 [784, 256, 256, 10] 형태의 DNN 이라고 생각하면 된다. 
Cost Function 은 앞에서 설명했던 Cross_entropy (Cost Function은 예측값과 Lable 값의 차이가 클수로 큰 값을 잘 리턴하면 된다)  
하지만 지금까지와는 달리 DNN 이기 때문에 Weight값을 Train 하기 위해서는 BackPropagation(편미분, Chain Rule) 을 사용해야 한다.     
Optimizer 는 지금까지는 우리가 잘 알고 있는 Gradient Descendent Algorithm 을 사용하였으나, 여기서는 AdamOptimizer라는 것을 사용한다.  
자세한 설명은 (참조 :  https://arxiv.org/pdf/1206.5533.pdf) 하면 되고, 간단하게 말하면 Gradient Descent Algorithm 보다 향상된 알고리즘이라고 생각하면 된다. (Other Optimizers :  http://sebastianruder.com/optimizing-gradient-descent/)

In [2]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Hidden layer with RELU activation
layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
layer_1 = tf.nn.relu(layer_1)
# Hidden layer with RELU activation
layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
layer_2 = tf.nn.relu(layer_2)
# Output layer with linear activation
pred = tf.matmul(layer_2, weights['out']) + biases['out']

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

<H3>(3) Session 실행</H3>   
실행은 특별한 것이 없다. (생략) 

In [3]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch: 0001 cost= 155.303240134
Epoch: 0002 cost= 38.519128221
Epoch: 0003 cost= 24.045136321
Epoch: 0004 cost= 16.552363712
Epoch: 0005 cost= 11.969245189
Epoch: 0006 cost= 8.740482058
Epoch: 0007 cost= 6.473937623
Epoch: 0008 cost= 4.765017819
Epoch: 0009 cost= 3.632678614
Epoch: 0010 cost= 2.667685981
Epoch: 0011 cost= 1.977117922
Epoch: 0012 cost= 1.585891090
Epoch: 0013 cost= 1.156536647
Epoch: 0014 cost= 0.948606044
Epoch: 0015 cost= 0.734226249
Optimization Finished!
Accuracy: 0.9462
